# References
- [Lora for Diffusers](https://github.com/haofanwang/Lora-for-Diffusers#full-model)
- [Long Prompts](https://huggingface.co/docs/diffusers/using-diffusers/custom_pipeline_examples#long-prompt-weighting-stable-diffusion)

# Usage
Save your `.safetensors` files in the `_models` so you can load them with this notebook. See the `Gen model` command!

# Config

## Install dependencies

In [ ]:
#! git clone https://github.com/huggingface/diffusers
#! pip install -U git+https://github.com/huggingface/diffusers
! git clone https://github.com/AguilarLagunasArturo/diffusers.git
! pip install -U git+https://github.com/AguilarLagunasArturo/diffusers.git
! pip install omegaconf
! pip install transformers safetensors accelerate

## Imports

In [2]:
import os
import torch
import random
import datetime
from PIL import Image

## Paths

In [3]:
drive_root = '/content/drive/MyDrive'
root_path = os.path.join(drive_root, 'Stable Diffusion')

models_folder = '_models'
models_path = os.path.join(root_path, models_folder)

text2image_folder = 'text2image'
text2image_path = os.path.join(root_path, text2image_folder)

# Functions

In [4]:
def imgSave(image, path):
  os.makedirs(path, exist_ok=True)
  dt = datetime.datetime.now()
  image.save(
      os.path.join(
          path,
          f'{dt.year}-{dt.month}-{dt.year}_{dt.hour}-{dt.minute}-{dt.second}.png'
    )
  )

# Gen model
- [X] safetensor
- [X] ckpt
- [ ] vae

In [ ]:
extensions = ['safetensors', 'ckpt']
available_models = []
loadable_models = []

for model_file in os.listdir(models_path):
    ext = model_file.split('.')[-1]
    if ext.lower() not in extensions:
        loadable_models.append(model_file)
        continue
    available_models.append(model_file)

model_to_load = ''

if len(available_models) > 0:
    for i, model_name in enumerate(available_models):
        print(f'[+] Model {i+1}: {model_name}')
    model_id = available_models[0]
    current_model_path = os.path.join(models_path, model_id)
    
    ext = model_id.split('.')[-1]
    model_dump = model_id.replace('.' + ext, '')
    print(f'[+] Model dump name: {model_dump}')

    if ext == 'safetensors':
        ! python ./diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path '{current_model_path}' --dump_path '{model_dump}' --from_safetensors
    elif ext == 'ckpt':
        ! python ./diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path '{current_model_path}' --dump_path '{model_dump}'
    else:
        # vae
        # ! cd ./diffusers; python ./scripts/convert_lora_safetensor_to_diffusers.py --checkpoint_path '{current_model_path}'  --dump_path '../model_dir' --from_safetensors
        raise Exception(f'Extension {ext} not supported')
    model_to_load = model_dump
else:
    print("[!] There's no available models to generate, selecting one already generated")
    print("\n[+] Models:")
    for i, model_name in enumerate(loadable_models):
        print(f'\t[+] {i+1}: {model_name}')
    model_to_load = os.path.join(models_path, loadable_models[-1])

print(f'\n[+] Model to load: {model_to_load}')
model_name = model_to_load

# Load model

In [ ]:
# from diffusers import StableDiffusionPipeline
# pipeline = StableDiffusionPipeline.from_pretrained('UberRealistic',torch_dtype=torch.float32)

In [ ]:
# from diffusers import DPMSolverMultistepScheduler

# pipeline = pipeline.to("cuda")
# pipeline.safety_checker = None
# pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
from diffusers import DiffusionPipeline
import torch

# Override model to load
# model_to_load = '/content/drive/MyDrive/Stable Diffusion/_models/AbysOrangeMix2'

print(f'[+] Loading model: {model_to_load}\n')

pipeline = DiffusionPipeline.from_pretrained(
    model_to_load,
    custom_pipeline="lpw_stable_diffusion",
    torch_dtype=torch.float16
)

In [24]:
pipeline = pipeline.to("cuda")
pipeline.safety_checker = None

In [25]:
from diffusers import DPMSolverMultistepScheduler
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [ ]:
#vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to("cuda")
#pipeline.vae = vae

# Upscaling

In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git

In [ ]:
from RealESRGAN import RealESRGAN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = RealESRGAN(device, scale=4)
# model.load_weights('weights/RealESRGAN_x4.pth', download=True)

model_scale = "4" # ["2", "4", "8"]
model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth', download=True)

# up_img = model.predict(images[0])
# imgSave(up_img)

# Main

## Text2Img

In [ ]:
seed = int(random.random()*100000)

prompts = [
    '',
]
negative = [
    '',
]

p = random.choice(prompts)
n = random.choice(negative)

# Override test prompts
p = ''
n = ''

print(p)
print(n)

generator = torch.Generator(device="cuda").manual_seed(seed)

img = pipeline(
    prompt=p,
    negative_prompt=n,
    num_inference_steps=80,
    guidance_scale=8,
    generator=generator,
    width = 512, # 768
    height = 768, # 512
    max_embeddings_multiples = 5,
).images[0]
img

In [35]:
imgSave(model.predict(img), text2image_path)

In [ ]:
seed = int(random.random()*100000)

prompts = [
    '',
]
negative = [
    '',
]

total = 80
for i in range(total):
    seed = int(random.random()*100000)

    index = random.choice([0,1])
    # p = prompts[index]
    # n = negative[index]
    
    p = random.choice(prompts)
    n = random.choice(negative)

    generator = torch.Generator(device="cuda").manual_seed(seed)
    img = pipeline(
        prompt=p,
        negative_prompt=n,
        num_inference_steps=80,
        guidance_scale=8,
        generator=generator,
        width = 512, # 512
        height = 768, # 768
        max_embeddings_multiples=5
    ).images[0]    
    imgSave(model.predict(img), text2image_path)
    n = i+1
    print(f"[+] Progress {n}/{total}: {((n)/total)*100:.1f}%")

In [38]:
from google.colab import runtime
runtime.unassign()